# "Historical" Tweets - from Twitter Search API using Tweepy Library <br>

_Authors: Amy Taylor and Veronica Giannotta_ <br> <br>
Preliminary steps before using Twitter's API:
1. Sign-up for a twitter account
2. Register a twitter developer account (requires email or phone number)
3. Create a developer app (I went with the name BlockedRoads)
4. Obtain your 'Access token' and 'access token secret' in the developer dashboard

Load Imports

In [7]:
import pandas as pd
import tweepy
import shutil
import jsonpickle
import json
import datetime
import config

from pandas.io.json import json_normalize

## Part 1: Prepare API credentials
**Step 1: Authenticate account and tokens through tweepy**
<br>Required credentials:
<br>`config.consumer_key` = 'your_consumer_key'
<br>`config.consumer_secret` = 'your_consumer_secret'
<br>`config.access_token` = 'your_access_token'
<br>`config.access_token_secret` = 'your_access_token_secret'

In [8]:
# authenticate account with tweepy
auth = tweepy.OAuthHandler('1ZWD1JS7xivKADeOCV7zVXgI4', 'TgsgO0MaUYw2dccUVC7KtD8SYUX4QYaoxT52D4ye2146d0sLHo')
auth.set_access_token('3799487133-rGUwhJRC2pKZmx8UbureNKPxXSdZAgsWnOrjMQk', 'jOr0vlL0gzyRh1gOQTievE2ZAvuN1sD2FgYCGilFKmE60')

# create API object to pull data from twitter - and pass in authentication code
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Now we are free to make Twitter API calls

**Step 2: Verify API is working with your account**
- Run this cell to check if program is working. Output is your twitter name

In [9]:
user = api.me()
print (user.name)

Dali Souayah


**Step 3: Use geo_search API to get the `place_id` for a particular location**
- `place_id` is a unique id created by Twitter that is assigned to different neighborhoods/cities/countries
- In the query parameter, type the name of the city or country you want the place_id for
- granularity = neighborhood (default) , city , admin or country
    - EX: when query = 'Boston', the place_id generated is the same for neighborhood or city ( but empty for country)

In [13]:
#  Determine the city, country, or location you want
places = api.geo_search(query="Medford, MA", 
                        granularity="city"
                                          )
place_id = places[0].id
print('place_id is: 988257060feb2e4b',place_id)

place_id is: 988257060feb2e4b 7520fc0be21c62bf


***List of search queries made:***
<br>NOTE: there must be no space between place and place_ID

    
|searchQuery| place ID |granularity| file | # of tweets| extended tweet? |
|---|---|---|---|---|---|
|'place:96683cc9126741d1 road closed' | USA |country| ./PoGo_USA_Tutorial.json20190114_7_1_36.json | 142 | - |
| 'place:1c69a67ad480e1b1 road closed' | Houston | city/neigh | ./historical_20190114_11_59.json| 15|- |
| 'Houston road closed' | NA | NA | ./historical_20190114_12_2.json | 9|- |
|'place:96683cc9126741d1 road closed' | USA |city| ./historical_20190115_14_57.json | 210 | yes |
|'place:1c69a67ad480e1b1 road closed' | Houston |city| ./historical_20190117_11_49.json | 13 | no |


## Part 2: Download Archived Tweets with the Search API
**Step 1: Use tweepy library to download tweets from Search API based on our query search terms, and save tweets as a json file**
 - Include a place ID in the query if necessary
 - Other search params that are optional: since='2019-01-03',until='2019-01-11'
 
useful resource: http://www.dealingdata.net/2016/07/23/PoGo-Series-Tweepy/

In [21]:
searchQuery = 'place:988257060feb2e4b closed traffic'

tweetCount = 0

#Open a text file to save the tweets to
with open('./medford.json', 'w') as f:

    #Tell the Cursor method: we want to use Search API (api.search), and our query
    for tweet in tweepy.Cursor(api.search,q=searchQuery,
#                                tweet_mode='extended'    #comment out extended tweet if desired
                              ).items() :         

        #Verify the tweet has place info before writing 
        if tweet.place is not None:
            
            #Write the JSON format to the text file, and add one to the number of tweets we've collected
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 40 tweets


**Step 2: Append a timestamp to the end of the json file name**

In [22]:
def file_conversion():
    #create a timestamp
    now = datetime.datetime.now()
    month = '0'+str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    now_str = str(now.year)+month+day+'_'+hour+'_'+minute
    
    
    #replace the name of our file with a new timestamped filename
    
    dest = './medford_' + now_str + ".json"
    shutil.move('./medford.json', dest)
    
    with open(dest, "r") as f:
        status = f.readlines()
        jsons = []
        for ind in status:
            jsons.append(json.loads(ind))
    return jsons

# Uncomment out the file_conversion function to activate the file name change
file_conversion()

[{'contributors': None,
  'coordinates': {'coordinates': [-71.07923, 42.34756], 'type': 'Point'},
  'created_at': 'Mon Apr 15 19:03:54 +0000 2019',
  'entities': {'hashtags': [{'indices': [19, 26], 'text': 'Boston'},
    {'indices': [81, 89], 'text': 'traffic'}],
   'symbols': [],
   'urls': [{'display_url': 'bit.ly/YAC1Ic',
     'expanded_url': 'http://bit.ly/YAC1Ic',
     'indices': [90, 113],
     'url': 'https://t.co/AoTr5OsSgQ'}],
   'user_mentions': []},
  'favorite_count': 2,
  'favorited': False,
  'geo': {'coordinates': [42.34756, -71.07923], 'type': 'Point'},
  'id': 1117866152829448192,
  'id_str': '1117866152829448192',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'place': {'attributes': {},
   'bounding_box': {'coordinates': [[[-71.19142

**Step 3: Add the newly created json file to the list below for easy access later**

In [23]:
# Previously created json files can be accessed from this list
json_df = pd.read_json("./medford_20190415_17_41.json", lines = True)

# Part 3: Explore Tweets from json file
### Option A: Tweets --> json file --> dataframe

In [24]:
# list of columns currently available in our dataframe
json_df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'favorite_count', 'favorited', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata',
       'place', 'possibly_sensitive', 'retweet_count', 'retweeted', 'source',
       'text', 'truncated', 'user'],
      dtype='object')

**View all tweets from the `text` column** (for the first 7 tweets)

In [28]:
json_df

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,lang,metadata,place,possibly_sensitive,retweet_count,retweeted,source,text,truncated,user
0,NaN,None,2019-04-11 12:55:40,"{'hashtags': [], 'symbols': [], 'urls': [], 'u...",0,False,None,1116323934675320833,1116323934675320832,NaN,...,en,"{'iso_language_code': 'en', 'result_type': 're...","{'attributes': {}, 'bounding_box': {'coordinat...",NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",Centennial Avenue from Campbell Ave to Northsh...,False,"{'contributors_enabled': False, 'created_at': ..."
1,NaN,None,2019-04-11 12:39:00,"{'hashtags': [], 'symbols': [], 'urls': [], 'u...",0,False,None,1116319736999157762,1116319736999157760,NaN,...,en,"{'iso_language_code': 'en', 'result_type': 're...","{'attributes': {}, 'bounding_box': {'coordinat...",NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",Wave Ave from Atlantic Ave to Dolphin Ave is c...,False,"{'contributors_enabled': False, 'created_at': ..."
2,NaN,"{'coordinates': [-71.08259, 42.40478], 'type':...",2019-04-07 15:45:31,"{'hashtags': [{'indices': [30, 38], 'text': 'M...",0,False,"{'coordinates': [42.40478, -71.08259], 'type':...",1114917126802948097,1114917126802948096,NaN,...,en,"{'iso_language_code': 'en', 'result_type': 're...","{'attributes': {}, 'bounding_box': {'coordinat...",0.0,0,False,"<a href=""http://www.sigalert.com/Map.asp?regio...",Road closed intermittently in #Medford on Rt-1...,False,"{'contributors_enabled': False, 'created_at': ..."


In [29]:
json_df['text']

0    Centennial Avenue from Campbell Ave to Northsh...
1    Wave Ave from Atlantic Ave to Dolphin Ave is c...
2    Road closed intermittently in #Medford on Rt-1...
Name: text, dtype: object

Explore attributes of the json file

In [20]:
def PrintMembers(obj):
    for attribute in dir(obj):
        #We don't want to show built in methods of the class
        if not attribute.startswith('__'):
            print(attribute)
PrintMembers(tweet)

_api
_json
author
contributors
coordinates
created_at
destroy
entities
favorite
favorite_count
favorited
geo
id
id_str
in_reply_to_screen_name
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
is_quote_status
lang
metadata
parse
parse_list
place
possibly_sensitive
retweet
retweet_count
retweeted
retweets
source
source_url
text
truncated
user


**Shrink dataframe down to only the useful columns**

In [38]:
columns = ['coordinates', 'created_at', 'geo', 'place', 'text', 'user']
df = json_df.loc[:, columns]
df.head()

,coordinates,created_at,geo,place,text,user
0,"{'coordinates': [-106.65, 35.08653], 'type': '...",2019-01-14 14:56:55,"{'coordinates': [35.08653, -106.65], 'type': '...",{'bounding_box': {'coordinates': [[[-106.79169...,"Road construction, left lane closed in #Albuqu...","{'listed_count': 73, 'friends_count': 226, 'pr..."
1,"{'coordinates': [-111.0295, 32.29418], 'type':...",2019-01-14 14:31:30,"{'coordinates': [32.29418, -111.0295], 'type':...",{'bounding_box': {'coordinates': [[[-111.08321...,Road construction. right lanes closed in #Pima...,"{'listed_count': 82, 'friends_count': 303, 'pr..."
2,"{'coordinates': [-106.52, 31.79295], 'type': '...",2019-01-14 14:28:12,"{'coordinates': [31.79295, -106.52], 'type': '...",{'bounding_box': {'coordinates': [[[-106.63487...,"Road construction, shoulder closed in #ElPaso ...","{'listed_count': 47, 'friends_count': 455, 'pr..."
3,None,2019-01-14 14:08:05,None,{'bounding_box': {'coordinates': [[[-86.044756...,Ughhh at the dentist for a cleaning and the si...,"{'listed_count': 37, 'friends_count': 1991, 'p..."
4,"{'coordinates': [-111.0295, 32.29418], 'type':...",2019-01-14 13:49:35,"{'coordinates': [32.29418, -111.0295], 'type':...",{'bounding_box': {'coordinates': [[[-111.08321...,Road constructions. two right lanes closed in ...,"{'listed_count': 82, 'friends_count': 303, 'pr..."


### Explore some features of the df

In [39]:
# see all the nested information within place (for the first two tweets)
list = json_df.loc[:1, 'place']
for i in range(len(list)):
    print(i, list[i])
    print("--------")

0 {'bounding_box': {'coordinates': [[[-106.7916912, 35.0158912], [-106.473745, 35.0158912], [-106.473745, 35.218114], [-106.7916912, 35.218114]]], 'type': 'Polygon'}, 'id': '813a485b26b8dae2', 'country_code': 'US', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/813a485b26b8dae2.json', 'contained_within': [], 'name': 'Albuquerque', 'attributes': {}, 'full_name': 'Albuquerque, NM', 'country': 'United States'}
--------
1 {'bounding_box': {'coordinates': [[[-111.083219, 32.057802], [-110.747928, 32.057802], [-110.747928, 32.320979], [-111.083219, 32.320979]]], 'type': 'Polygon'}, 'id': '013379ee5729a5e6', 'country_code': 'US', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/013379ee5729a5e6.json', 'contained_within': [], 'name': 'Tucson', 'attributes': {}, 'full_name': 'Tucson, AZ', 'country': 'United States'}
--------


> The dataframe that we have now has many attributes nested within other attributes. If we want to access the corrdinates nested within `coordinates` or the city nested within `place` we will need to unpack these variables using a different method. Use the next section to convert nested values into their own columns.

## Option B. Tweets --> json --> parsed --> nested dataframe

In [46]:
# Read in json file
json_df = pd.read_json("../data/AT_historical/PoGo_USA_Tutorial.json20190114_7_1_36.json", lines = True)

# convert json file to list of dictionaries
tweets_data = []
notParsed = []
tweets_file = open("../data/AT_historical/PoGo_USA_Tutorial.json20190114_7_1_36.json","r")
for line in tweets_file:    
    if line.strip():    
        try:
            tweet=json.loads(line)
            tweets_data.append(tweet)
        except:
            notParsed.append(line)
            continue
print(len(tweets_data))
print('Could not parse: ', len(notParsed))

143
Could not parse:  0


**Unpack the nested columns**

In [47]:
tweet_cols = ['coordinates', 'created_at', 
#                'full_text',
              'text','geo', 'id', 'place', 'user']

sample_tweets_dict = [{col:tweet[col] for col in tweet_cols } for tweet in tweets_data]

# This method looks for any instances of nested dictionaries (DOES NOT FLATTEN LISTS)
sample_tweets_df = pd.io.json.json_normalize(sample_tweets_dict)

sample_tweets_df.head(2)

,coordinates,coordinates.coordinates,coordinates.type,created_at,geo,geo.coordinates,geo.type,id,place.bounding_box.coordinates,place.bounding_box.type,...,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.translator_type,user.url,user.utc_offset,user.verified
0,NaN,"[-106.65, 35.08653]",Point,Mon Jan 14 14:56:55 +0000 2019,NaN,"[35.08653, -106.65]",Point,1084826703724249089,"[[[-106.7916912, 35.0158912], [-106.473745, 35...",Polygon,...,333333,True,False,TotalTrafficABQ,22351,None,none,https://t.co/0ksDF2WEvB,None,False
1,NaN,"[-111.0295, 32.29418]",Point,Mon Jan 14 14:31:30 +0000 2019,NaN,"[32.29418, -111.0295]",Point,1084820307217731584,"[[[-111.083219, 32.057802], [-110.747928, 32.0...",Polygon,...,333333,True,False,TotalTrafficTUC,44176,None,none,None,None,False


In [48]:
sample_tweets_df.columns

Index(['coordinates', 'coordinates.coordinates', 'coordinates.type',
       'created_at', 'geo', 'geo.coordinates', 'geo.type', 'id',
       'place.bounding_box.coordinates', 'place.bounding_box.type',
       'place.contained_within', 'place.country', 'place.country_code',
       'place.full_name', 'place.id', 'place.name', 'place.place_type',
       'place.url', 'text', 'user.contributors_enabled', 'user.created_at',
       'user.default_profile', 'user.default_profile_image',
       'user.description', 'user.entities.description.urls',
       'user.entities.url.urls', 'user.favourites_count',
       'user.follow_request_sent', 'user.followers_count', 'user.following',
       'user.friends_count', 'user.geo_enabled', 'user.has_extended_profile',
       'user.id', 'user.id_str', 'user.is_translation_enabled',
       'user.is_translator', 'user.lang', 'user.listed_count', 'user.location',
       'user.name', 'user.notifications', 'user.profile_background_color',
       'user.profile_bac

In [43]:
# save the USA (or Houston) historical json file as df, COMMENT OUT
# df = sample_tweets_df
# df.to_csv("../data/AT_historical/USA_142tweets.csv", index = False)

Examine who the most common tweets are coming from

In [49]:
sample_tweets_df['user.screen_name'].value_counts().head()

TotalTrafficDFW    16
TotalTrafficABQ    13
TotalTrafficMIA    10
TotalTrafficPHX     8
AlgoTraffic         7
Name: user.screen_name, dtype: int64

So the Twitter user "TotalTraffic", specific to a certain city, is contributing the most tweets to situations with "road closed", and providing location place data for each tweet